# Lambda³ Zero-Shot Anomaly Detection Demo

This notebook demonstrates the revolutionary Lambda³ anomaly detection system that achieves 93% AUC without any training data.

In [ ]:
# Cell 1: Installation and Setup
!pip install numpy numba scikit-learn scipy matplotlib pandas

# Clone or download the detector
!wget https://raw.githubusercontent.com/yourusername/lambda3-anomaly-detection/main/lambda3_detector.py

In [ ]:
# Cell 2: Import and Basic Usage
import numpy as np
from lambda3_detector import (
    Lambda3ZeroShotDetector, 
    L3Config,
    create_complex_natural_dataset
)

# Generate test data
events, labels, anomaly_types = create_complex_natural_dataset(
    n_events=500,
    n_features=15,
    anomaly_ratio=0.15
)
print(f"Data shape: {events.shape}")
print(f"Anomaly ratio: {np.mean(labels):.2%}")
print(f"Anomaly types: {set(anomaly_types)}")

In [ ]:
# Cell 3: Run Detection
# Initialize detector with optimized config
config = L3Config(
    alpha=0.05,
    beta=0.005,
    n_paths=7,
    jump_scale=1.5,
    w_topo=0.3,
    w_pulse=0.2
)
detector = Lambda3ZeroShotDetector(config)

# Analyze (no training needed!)
result = detector.analyze(events)

# Detect anomalies
anomaly_scores = detector.detect_anomalies(result, events, use_adaptive_weights=False)

# Calculate performance
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(labels, anomaly_scores)
print(f"Zero-shot AUC: {auc:.4f}")

In [ ]:
# Cell 4: Visualize Results
import matplotlib.pyplot as plt

# Plot anomaly scores with better visualization
plt.figure(figsize=(15, 6))

# Subplot 1: Time series view
plt.subplot(2, 1, 1)
plt.plot(anomaly_scores, 'b-', alpha=0.7, linewidth=1, label='Anomaly Score')
plt.scatter(np.where(labels)[0], anomaly_scores[labels==1], 
           color='red', s=50, alpha=0.8, label='True Anomalies', zorder=5)
plt.axhline(y=2.0, color='orange', linestyle='--', alpha=0.5, label='Warning Threshold')
plt.xlabel('Event Index')
plt.ylabel('Anomaly Score')
plt.title(f'Lambda³ Zero-Shot Detection (AUC: {auc:.3f})')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Score distribution
plt.subplot(2, 1, 2)
plt.hist(anomaly_scores[labels==0], bins=50, alpha=0.7, label='Normal', density=True)
plt.hist(anomaly_scores[labels==1], bins=50, alpha=0.7, label='Anomaly', density=True)
plt.xlabel('Anomaly Score')
plt.ylabel('Density')
plt.title('Score Distribution by Class')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Cell 5: Advanced Usage with All Modes
# Basic mode (already computed above)
print(f"Basic Mode AUC: {auc:.4f}")

# Adaptive mode
adaptive_scores = detector.detect_anomalies(result, events, use_adaptive_weights=True)
adaptive_auc = roc_auc_score(labels, adaptive_scores)
print(f"Adaptive Mode AUC: {adaptive_auc:.4f}")

# Focused mode
from lambda3_detector import Lambda3FocusedDetector
focused_detector = Lambda3FocusedDetector()
focused_scores = focused_detector.detect_anomalies(
    result, events, 
    base_detector_func=lambda r, e: detector.detect_anomalies(r, e)
)
focused_auc = roc_auc_score(labels, focused_scores)
print(f"Focused Mode AUC: {focused_auc:.4f}")

# Top anomalies analysis
best_scores = anomaly_scores if auc >= adaptive_auc else adaptive_scores
top_10 = np.argsort(best_scores)[-10:]
print(f"\nTop 10 detected anomalies: {top_10}")
print(f"Precision@10: {np.sum(labels[top_10])}/10 = {np.mean(labels[top_10]):.1%}")

# Get explanations for top anomalies
print("\nTop 3 Anomaly Explanations:")
for i, idx in enumerate(top_10[-3:][::-1]):
    explanation = detector.explain_anomaly(idx, result, events)
    print(f"\n{i+1}. Event {idx}:")
    print(f"   Score: {best_scores[idx]:.3f}")
    print(f"   True Label: {'Anomaly' if labels[idx] else 'Normal'}")
    print(f"   {explanation['recommendation']}")

In [ ]:
# Cell 6: Full Visualization (Optional)
# Generate comprehensive visualization
fig = detector.visualize_results(events, result, best_scores)
plt.show()